In [1]:
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

In [2]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer
model = AutoPeftModelForCausalLM.from_pretrained(
    "ahmedembedded/AskFAST",
    load_in_4bit = load_in_4bit,
)
tokenizer = AutoTokenizer.from_pretrained("ahmedembedded/AskFAST")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:167: UserWarning: You passed `quantization_config` 

In [3]:
FAST_prompt_context = """You are an admission officer at Fast University Pakistan. Your role is to answer queries related to the admission process at Fast University. You are expected to provide detailed and accurate responses to questions regarding:

- Application deadlines
- Required documents
- Eligibility criteria for different programs
- Admission process details
- Any other admissions-related information specific to Fast University Pakistan

Do not respond to any questions that are not related to admissions at Fast University Pakistan. Maintain a professional and helpful tone, ensuring that prospective students receive the information they need to apply successfully. If there's a question about comparisons, respond "I'm not a career counselling bot".

**Example Questions:**

1. What is the application deadline for the upcoming semester?
2. What documents are required for the application?
3. What are the eligibility criteria for the Computer Science program?
4. How competitive is the admission process for the Business Administration program?

Stay focused on admissions-related topics only.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


In [6]:
past_prompts = []

def get_answer(question: str) -> str:
    if len(past_prompts) >= 10:
        past_prompts.pop(0)

    past_prompts.append(f"User: {question}")

    inputs = tokenizer(
        [
            FAST_prompt_context.format(
                past_prompts,
                question,
                "",
            )
        ], return_tensors="pt").to("cuda")

    response = model.generate(**inputs, max_new_tokens=128)

    response_text = tokenizer.decode(response[0], skip_special_tokens=True)

    past_prompts.append(question)
    past_prompts.append(response_text.split('Response:')[1].split('### Input:')[0])

    return response_text.split('Response:')[1].split('### Input:')[0]


In [8]:
get_answer("How's Lahore weather?")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


"\nI'm sorry, I don't have the information on Lahore's weather. You may want to check local news reports or websites dedicated to weather information."

In [9]:
import subprocess
subprocess.run(['pip', 'install', 'anvil-uplink'])
import anvil.server
import anvil.mpl_util

NameError: name 'subprocess' is not defined

In [ ]:
anvil.server.connect("server_4A4U4BE7XQDAEMCUOC3JW7ZY-5ACGOW4HWMXBB2OB")
prompts_daily = [12, 45, 80, 1]

@anvil.server.callable
def get_response_from_model(input_text):
    global prompts_daily
    prompts_daily[-1] += 1
    return get_answer(input_text)

@anvil.server.callable
def make_plot():
    global prompts_daily
    plt.figure(figsize=(20, 5), facecolor='white')
    plt.bar(range(len(prompts_daily)), prompts_daily, color='#668bd6')
    plt.xlabel('Index')
    plt.ylabel('Value')
    plt.title('Prompts Daily')
    plt.gca().set_facecolor('white')
    return anvil.mpl_util.plot_image()

anvil.server.wait_forever()